In [338]:
##########define states and constants########
gamma = 1.0

#variable 1
grid_length = 100

#variable 2
gas_level_interval = 0.1
num_gas_level_intervals = int(1 / gas_level_interval)

#variable 3
gas_station_present = 2

#variable 4
gas_price_max = 4.00
gas_price_min = 3.00
gas_price_interval = 0.10
num_gas_price_intervals = int((gas_price_max - gas_price_min) / gas_price_interval)

total_num_states = grid_length * grid_length * num_gas_level_intervals * gas_station_present * num_gas_price_intervals

##########define actions########

#move left, right, up, down
num_moves = 4
#pump or not pump
pump_or_not = 1

total_actions = num_moves + pump_or_not

#########initialization########

#create a 4d matrix, change it to sparse matrix
R = zeros(grid_length,grid_length,num_gas_level_intervals,gas_station_present,num_gas_price_intervals, total_actions)

#randomly pick starting point
start_pt = (int(rand() * grid_length) , int(rand() * grid_length))

#randomly pick ending point
end_pt = (int(rand() * grid_length) , int(rand() * grid_length))
println("end: ", end_pt)

#randomly intialize gas stations
num_gas_stations = 15
station_set = Set()
for i = 1:num_gas_stations
    station_pt = (int(rand() * grid_length) , int(rand() * grid_length))
    push!(station_set,station_pt)
end
println("gas: ", station_set)


########initialize rewards#######

#initialize destination reward
destination_default_reward = 100
dest_offset = 50
for i = 1:num_gas_level_intervals
    gas_level = (i / num_gas_level_intervals)
    R[end_pt[1],end_pt[2],i,:,:,:] = destination_default_reward + dest_offset * gas_level
end

#initialize deadend reward
dead_end_reward = -1000
R[:,:, 1,2,:,:] = dead_end_reward
#edge case - ensure u can finsih at end point with 0 gas left
gas_level = (1 / num_gas_level_intervals)
R[end_pt[1],end_pt[2],1,2,:,:] = destination_default_reward + dest_offset * gas_level

#initialize refuelling reward
refuel_default_reward = 20
for i = 1:num_gas_level_intervals
    for j = 1:num_gas_price_intervals
        gas_level = (i / num_gas_level_intervals)
        curr_price = gas_price_min + (j / num_gas_price_intervals) * (gas_price_max - gas_price_min)
        #action 5 is refuel, 1 means gas station is present
        for pt in station_set
            R[pt[1],pt[2],i,1,j,5] = refuel_default_reward * (1 - gas_level) / curr_price
            #if gas is already full, why fill it again?
            if gas_level == 1
                R[pt[1],pt[2],i,1,j,5] = -100
            end
        end
#         R[:,:,i,1,j,5] = refuel_default_reward * (1 - gas_level) / curr_price
#         #if gas is already full, why fill it again?
#         if gas_level == 1
#             R[:,:,i,1,j,5] = -100
#         end
    end
end


end: (18,93)
gas: Set{Any}({(67,74),(75,82),(30,53),(65,81),(52,91),(35,17),(45,81),(98,65),(12,61),(69,72),(96,24),(39,83),(28,90),(6,65),(44,73)})


In [339]:
#########define transtion functions##########

function get_next_states(x, y, num_gas_level_intervals,gas_station_present,num_gas_price_intervals)
    
    next_state_array = Tuple[]
    
    #move up down left right
    if x + 1 <= grid_length && num_gas_level_intervals - 1 >= 1
        push!(next_state_array, (x + 1, y, num_gas_level_intervals - 1, gas_station_present,num_gas_price_intervals))
    end
    
    if x - 1 >= 1 && num_gas_level_intervals - 1 >= 1
        push!(next_state_array, (x - 1, y, num_gas_level_intervals - 1, gas_station_present,num_gas_price_intervals))
    end
    
    if y + 1 <= grid_length && num_gas_level_intervals - 1 >= 1
        push!(next_state_array, (x, y + 1, num_gas_level_intervals - 1, gas_station_present,num_gas_price_intervals))
    end
    
    if y - 1 >= 1 && num_gas_level_intervals - 1 >= 1
        push!(next_state_array, (x, y - 1, num_gas_level_intervals - 1, gas_station_present,num_gas_price_intervals))
    end
    
    #topping up gas
    if in((x,y), station_set)
        push!(next_state_array, (x, y, 1, gas_station_present,num_gas_price_intervals))
    end
    
#     println((x, y, 1, gas_station_present,num_gas_price_intervals))
    
    return next_state_array
end


get_next_states (generic function with 2 methods)

In [340]:
#find all the reachable states from start state
tic()
U = zeros(grid_length,grid_length,num_gas_level_intervals,gas_station_present,num_gas_price_intervals)
A = zeros(grid_length,grid_length,num_gas_level_intervals,gas_station_present,num_gas_price_intervals)
for k = 1:10
    for i = 1:grid_length
        for j = 1:grid_length
            for glevel = 1:num_gas_level_intervals
                for gpresent = 1:gas_station_present
                    for plevel = 1:num_gas_price_intervals
                        U_max = -99999
                        for a = 1:total_actions
                            curr_r = R[i,j,glevel,gpresent,plevel,a]
                            next_states_array = get_next_states(i,j,glevel,gpresent,plevel)
                            accum_reward = 0
                            for s in next_states_array
                                accum_reward += R[s[1],s[2],s[3],s[4],s[5],a]
                            end
                            curr_u = curr_r + gamma * accum_reward
                            if curr_u > U_max
                                U_max = curr_u
                                A[i,j,glevel,gpresent,plevel] = a
                            end
                        end
                        U[i,j,glevel,gpresent,plevel] = U_max
                    end
                end
            end
        end
        if i%51 == 0
            println("half", i)
        end
    end
    println("k: ",k)
end
toc()

half51
k: 1
half51
k: 2
half51
k: 3
half51
k: 4
half51
k: 5
half51
k: 6
half51
k: 7
half51
k: 8
half51
k: 9
half51
k: 10
elapsed time: 173.924053985 seconds


173.924053985

In [128]:
(grid_length,grid_length,num_gas_level_intervals,gas_station_present,num_gas_price_intervals, total_actions), total_num_states

((100,100,10,2,10,5),2000000)

In [154]:
#begin value iteration

In [157]:
nextd_states = [1 2 3]

1x3 Array{Int64,2}:
 1  2  3

In [160]:
length(nextd_states)

3

In [221]:
R[end_pt[1],end_pt[2],:,:,:,:]

1x1x10x2x10x5 Array{Float64,6}:
[:, :, 1, 1, 1, 1] =
 105.0

[:, :, 2, 1, 1, 1] =
 110.0

[:, :, 3, 1, 1, 1] =
 115.0

[:, :, 4, 1, 1, 1] =
 120.0

[:, :, 5, 1, 1, 1] =
 125.0

[:, :, 6, 1, 1, 1] =
 130.0

[:, :, 7, 1, 1, 1] =
 135.0

[:, :, 8, 1, 1, 1] =
 140.0

[:, :, 9, 1, 1, 1] =
 145.0

[:, :, 10, 1, 1, 1] =
 150.0

[:, :, 1, 2, 1, 1] =
 105.0

[:, :, 2, 2, 1, 1] =
 110.0

[:, :, 3, 2, 1, 1] =
 115.0

[:, :, 4, 2, 1, 1] =
 120.0

[:, :, 5, 2, 1, 1] =
 125.0

[:, :, 6, 2, 1, 1] =
 130.0

[:, :, 7, 2, 1, 1] =
 135.0

[:, :, 8, 2, 1, 1] =
 140.0

[:, :, 9, 2, 1, 1] =
 145.0

[:, :, 10, 2, 1, 1] =
 150.0

[:, :, 1, 1, 2, 1] =
 105.0

[:, :, 2, 1, 2, 1] =
 110.0

[:, :, 3, 1, 2, 1] =
 115.0

[:, :, 4, 1, 2, 1] =
 120.0

[:, :, 5, 1, 2, 1] =
 125.0

[:, :, 6, 1, 2, 1] =
 130.0

[:, :, 7, 1, 2, 1] =
 135.0

[:, :, 8, 1, 2, 1] =
 140.0

[:, :, 9, 1, 2, 1] =
 145.0

[:, :, 10, 1, 2, 1] =
 150.0

[:, :, 1, 2, 2, 1] =
 105.0

[:, :, 2, 2, 2, 1] =
 110.0

[:, :, 3, 2, 2, 1] =
 115.0

[:, :, 4

In [342]:
U[end_pt[1],end_pt[2],:,:,:]

1x1x10x2x10 Array{Float64,5}:
[:, :, 1, 1, 1] =
 105.0

[:, :, 2, 1, 1] =
 110.0

[:, :, 3, 1, 1] =
 115.0

[:, :, 4, 1, 1] =
 120.0

[:, :, 5, 1, 1] =
 125.0

[:, :, 6, 1, 1] =
 130.0

[:, :, 7, 1, 1] =
 135.0

[:, :, 8, 1, 1] =
 140.0

[:, :, 9, 1, 1] =
 145.0

[:, :, 10, 1, 1] =
 150.0

[:, :, 1, 2, 1] =
 105.0

[:, :, 2, 2, 1] =
 -3890.0

[:, :, 3, 2, 1] =
 115.0

[:, :, 4, 2, 1] =
 120.0

[:, :, 5, 2, 1] =
 125.0

[:, :, 6, 2, 1] =
 130.0

[:, :, 7, 2, 1] =
 135.0

[:, :, 8, 2, 1] =
 140.0

[:, :, 9, 2, 1] =
 145.0

[:, :, 10, 2, 1] =
 150.0

[:, :, 1, 1, 2] =
 105.0

[:, :, 2, 1, 2] =
 110.0

[:, :, 3, 1, 2] =
 115.0

[:, :, 4, 1, 2] =
 120.0

[:, :, 5, 1, 2] =
 125.0

[:, :, 6, 1, 2] =
 130.0

[:, :, 7, 1, 2] =
 135.0

[:, :, 8, 1, 2] =
 140.0

[:, :, 9, 1, 2] =
 145.0

[:, :, 10, 1, 2] =
 150.0

[:, :, 1, 2, 2] =
 105.0

[:, :, 2, 2, 2] =
 -3890.0

[:, :, 3, 2, 2] =
 115.0

[:, :, 4, 2, 2] =
 120.0

[:, :, 5, 2, 2] =
 125.0

[:, :, 6, 2, 2] =
 130.0

[:, :, 7, 2, 2] =
 135.0

[

In [341]:
println(end_pt[1]," ",end_pt[2])
f = open("/Users/yishh/Box Sync/CS 238/project_stuff/proj_utility_output_1","w")
a_f = open("/Users/yishh/Box Sync/CS 238/project_stuff/proj_actions_output_1","w")
for i = 1:grid_length
    for j = 1:grid_length
        #get the max of each x,y coordinate
        #find the action
#         print(maximum(U[i,j,:,:,:]), " ")
        write(f, string(int(maximum(U[i,j,:,:,:]))))
        write(f, " ")
        write(a_f, string(A[i,j,:,:,:][indmax(U[i,j,:,:,:])]))
        write(a_f, " ")
    end
#     println("")
    write(f, "\n")
    write(a_f, "\n")
end
close(f)
close(a_f)

18 93


In [343]:
maximum(U[end_pt[1],end_pt[2],:,:,:])

150.0

In [344]:
indmax(U[end_pt[1],end_pt[2],:,:,:]) 

10

In [345]:
U[end_pt[1]-2,end_pt[2],:,:,:]

1x1x10x2x10 Array{Float64,5}:
[:, :, 1, 1, 1] =
 0.0

[:, :, 2, 1, 1] =
 0.0

[:, :, 3, 1, 1] =
 0.0

[:, :, 4, 1, 1] =
 0.0

[:, :, 5, 1, 1] =
 0.0

[:, :, 6, 1, 1] =
 0.0

[:, :, 7, 1, 1] =
 0.0

[:, :, 8, 1, 1] =
 0.0

[:, :, 9, 1, 1] =
 0.0

[:, :, 10, 1, 1] =
 0.0

[:, :, 1, 2, 1] =
 -1000.0

[:, :, 2, 2, 1] =
 -4000.0

[:, :, 3, 2, 1] =
 0.0

[:, :, 4, 2, 1] =
 0.0

[:, :, 5, 2, 1] =
 0.0

[:, :, 6, 2, 1] =
 0.0

[:, :, 7, 2, 1] =
 0.0

[:, :, 8, 2, 1] =
 0.0

[:, :, 9, 2, 1] =
 0.0

[:, :, 10, 2, 1] =
 0.0

[:, :, 1, 1, 2] =
 0.0

[:, :, 2, 1, 2] =
 0.0

[:, :, 3, 1, 2] =
 0.0

[:, :, 4, 1, 2] =
 0.0

[:, :, 5, 1, 2] =
 0.0

[:, :, 6, 1, 2] =
 0.0

[:, :, 7, 1, 2] =
 0.0

[:, :, 8, 1, 2] =
 0.0

[:, :, 9, 1, 2] =
 0.0

[:, :, 10, 1, 2] =
 0.0

[:, :, 1, 2, 2] =
 -1000.0

[:, :, 2, 2, 2] =
 -4000.0

[:, :, 3, 2, 2] =
 0.0

[:, :, 4, 2, 2] =
 0.0

[:, :, 5, 2, 2] =
 0.0

[:, :, 6, 2, 2] =
 0.0

[:, :, 7, 2, 2] =
 0.0

[:, :, 8, 2, 2] =
 0.0

[:, :, 9, 2, 2] =
 0.0

[:, :, 10, 2, 2

In [295]:
length(U[end_pt[1],end_pt[2],:,:,:])

200

In [ ]:
function convertIndex(ind, num_gas_level_intervals,gas_station_present,num_gas_price_intervals)
    remainder = ind / num_gas_level_intervals / gas_station_present
    remainder = remainder % num_gas_price_intervals
end